In [1]:
import numpy as np  # Für numerische Berechnungen und Matrixoperationen
import pandas as pd  # Für die Verarbeitung von Tabellen-Daten (DataFrames)
from scipy.sparse import csr_matrix, save_npz, load_npz, vstack  # Für sparsere Matrizen, Speicheroptimierung
from sklearn.cluster import DBSCAN  # DBSCAN für die Clusteranalyse

# Lade die transformierten Daten (erste 45.000 Punkte)
df_encoded = pd.read_csv("dbscan_encoded_data.csv").head(45000)  # Nur die ersten 45.000 Punkte werden verwendet

# Bestimme die Anzahl der Samples und die Batch-Größe
num_samples = len(df_encoded)  # Anzahl der Samples im Datensatz
batch_size = 100  # Größe jedes Batches, anpassbar je nach verfügbarem RAM
num_batches = (num_samples // batch_size) + 1  # Bestimmt, wie viele Batches benötigt werden
alpha = 0.5  # Gewichtung für geographische Distanz (Haversine) und euklidische Distanz

# Verarbeitung der Distanzmatrizen in Batches und Speichern als Sparse-Matrix (Speicheroptimierung)
for batch in range(num_samples):
    try:
        # Lade die Distanzmatrizen für das aktuelle Batch (Haversine und Euklidische Distanzen)
        geo_batch = np.load(f"haversine_batch_{batch}.npy").astype(np.float32)  # Haversine-Distanzmatrizen
        euc_batch = np.load(f"euklid_batch_{batch}.npy").astype(np.float32)  # Euklidische Distanzmatrizen
    except FileNotFoundError:
        print(f"Batch {batch} nicht gefunden, überspringe...")  # Wenn eine Datei nicht gefunden wird, wird das Batch übersprungen
        continue

    # Berechne den Indexbereich des aktuellen Batches
    start_idx = batch * batch_size
    end_idx = min((batch + 1) * batch_size, num_samples)

    # Berechnung der gewichteten Distanzmatrix
    batch_matrix = alpha * geo_batch + (1 - alpha) * euc_batch  # Kombiniere die Distanzen basierend auf dem Gewicht alpha

    # Speichern als sparse Matrix im CSR-Format (Komprimierung und Speicheroptimierung)
    save_npz(f"combined_distance_batch_{batch}.npz", csr_matrix(batch_matrix))  # Speichern der Matrix im .npz-Format

print("Alle Batches gespeichert!")


OSError: [Errno 28] No space left on device